# step1 : sra2fastq文件

In [ ]:
import subprocess
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

## 批量下载SRA数据
def prefetch_sra(sra_accession, work_dir):
    try:
        subprocess.run(['prefetch', '-O', work_dir, sra_accession], check=True)
        print(f"Prefetching {sra_accession} completed successfully.")
        return sra_accession
    except subprocess.CalledProcessError as e:
        print(f"Prefetching {sra_accession} failed with error: {e}")
        return None

## SRA转为fastq
def fastq_dump(sra_accession, work_dir, output_dir):
    sra_file = os.path.join(work_dir, sra_accession, f"{sra_accession}.sra")
    if not os.path.exists(sra_file):
        print(f"SRA file {sra_file} does not exist. Skipping fastq-dump.")
        return
    try:
        subprocess.run(['fastq-dump', '--gzip', '--split-3', '-O', output_dir, sra_file], check=True)
        print(f"Fastq-dump {sra_accession} completed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Fastq-dump {sra_accession} failed with error: {e}")

def process_sra(sra_accession, work_dir, output_dir):
    if prefetch_sra(sra_accession, work_dir):
        fastq_dump(sra_accession, work_dir, output_dir)

work_dir = "/home/john/RNA-seq/RNASeqDemoData/airway"
output_dir = '/home/john/RNA-seq/RNASeqDemoData/fastq'
max_workers = 10

with open("/home/john/RNA-seq/RNASeqDemoData/airway/sra_id", "r") as file:
    sra_ids = [line.strip() for line in file if line.strip()]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_sra, sra_id, work_dir, output_dir) for sra_id in sra_ids]
        for future in as_completed(futures):
            pass


# step2 : 质量检测 (fastqc + multiqc)

In [1]:

import subprocess
import os
import webbrowser

#每个fq单独质控
def run_fastqc(fastq_file, output_dir):
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    fastqc_command = ['fastqc', fastq_file, '-o', output_dir]
    ## FastQC
    try:
        subprocess.run(fastqc_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"FastQC has finished analyzing {fastq_file}")
        html_report_path = os.path.join(output_dir, os.path.basename(fastq_file) + '_fastqc.html')
        webbrowser.open('file://' + html_report_path )
        
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running FastQC on: {fastq_file}: {e}")

#总质控
def generate_multiqc_report(fastqczip_path, output_dir):
    multiqc_command = ['multiqc', fastqczip_path, '-o', output_dir]
    
    try:
        subprocess.run(multiqc_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("MultiQC report generated successfully.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while generated the MultiQC report:{e}")

fastq_path = "/home/john/ZouLab_Code/RNASeq/Rawdata/HeSR/"
fastq_files = [ff for ff in os.listdir(fastq_path) if ff.endswith('fq.gz')]
print(fastq_files)
fastqcAnal_path = "/home/john/ZouLab_Code/RNASeq/Fastqc/HeSR_fastqc/SigAnal"
multiqcAnal_path = "/home/john/ZouLab_Code/RNASeq/Fastqc/HeSR_fastqc/MulAnal"
# print(fastqc_zips)
for fq in fastq_files:
    fastq_file = fastq_path + fq
    run_fastqc(fastq_file, fastqcAnal_path)

generate_multiqc_report(fastqcAnal_path, multiqcAnal_path)

['HeSRD1_R1.fq.gz', 'HeSRA1_R2.fq.gz', 'HeSRT2_R1.fq.gz', 'HeSRT1_R2.fq.gz', 'HeSRD1_R2.fq.gz', 'HeSRA2_R2.fq.gz', 'HeSRD2_R1.fq.gz', 'HeSRT3_R2.fq.gz', 'HeSRD3_R2.fq.gz', 'HeSRT3_R1.fq.gz', 'HeSRA2_R1.fq.gz', 'HeSRD3_R1.fq.gz', 'HeSRT1_R1.fq.gz', 'HeSRD2_R2.fq.gz', 'HeSRA3_R2.fq.gz', 'HeSRA3_R1.fq.gz', 'HeSRA1_R1.fq.gz', 'HeSRT2_R2.fq.gz']
FastQC has finished analyzing /home/john/ZouLab_Code/RNASeq/Rawdata/HeSR/HeSRD1_R1.fq.gz


xdg-open: file '/home/john/ZouLab_Code/RNASeq/Fastqc/HeSR_fastqc/SigAnal/HeSRD1_R1.fq.gz_fastqc.html' does not exist


FastQC has finished analyzing /home/john/ZouLab_Code/RNASeq/Rawdata/HeSR/HeSRA1_R2.fq.gz


xdg-open: file '/home/john/ZouLab_Code/RNASeq/Fastqc/HeSR_fastqc/SigAnal/HeSRA1_R2.fq.gz_fastqc.html' does not exist


FastQC has finished analyzing /home/john/ZouLab_Code/RNASeq/Rawdata/HeSR/HeSRT2_R1.fq.gz


...

FastQC has finished analyzing /home/john/ZouLab_Code/RNASeq/Rawdata/HeSR/HeSRA1_R1.fq.gz


xdg-open: file '/home/john/ZouLab_Code/RNASeq/Fastqc/HeSR_fastqc/SigAnal/HeSRA1_R1.fq.gz_fastqc.html' does not exist


FastQC has finished analyzing /home/john/ZouLab_Code/RNASeq/Rawdata/HeSR/HeSRT2_R2.fq.gz


xdg-open: file '/home/john/ZouLab_Code/RNASeq/Fastqc/HeSR_fastqc/SigAnal/HeSRT2_R2.fq.gz_fastqc.html' does not exist


MultiQC report generated successfully.


# 过滤低质量数据 (trim_galore)

In [7]:
import os
import subprocess

# 定义工作目录和输出目录
work_dir = '/home/john/ZouLab_Code/RNASeq/Rawdata/HeSR'
output_dir = '/home/john/ZouLab_Code/RNASeq/TrimGaloreData/HeSR_val/'
os.makedirs(output_dir, exist_ok=True)

# 查找所有符合条件的双端测序文件
fq1_files = [file for file in os.listdir(work_dir) if 'HeSRD3_R1.fq.gz' in file]
fq2_files = [file for file in os.listdir(work_dir) if 'HeSRD3_R2.fq.gz' in file]
assert len(fq1_files) == len(fq2_files), "Number of R1 and R2 files do not match."

# 将文件对写入文件列表
file_pairs = '\n'.join(f'{fq1}\t{fq2}' for fq1, fq2 in zip(fq1_files, fq2_files))
with open('file_all', 'w') as f:
    f.write(file_pairs)

# 读取文件列表并处理每个文件对
# with open('file_all', 'r') as f:
#     for line in f:
#         fq1, fq2 = line.strip().split()
#         fq1_path = os.path.join(work_dir, fq1)
#         fq2_path = os.path.join(work_dir, fq2)
#         fq1_val_path = os.path.join(output_dir, fq1.replace('.fq.gz', '_cut.fq.gz'))
#         fq2_val_path = os.path.join(output_dir, fq2.replace('.fq.gz', '_cut.fq.gz'))
        
#         print(f"Processing: {fq1_path} and {fq2_path}")
        
#         # 构建 cutadapt 命令
#         cutadapt_command = [
#             'cutadapt',
#             '-a', 'AATG',  # Read1的3'端适配器
#             '-A', 'CAAGCAGAAGACGGCATACGAGAT',  # Read2的3'端适配器
#             '-g', 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC',  # Read1的5'端适配器
#             '-G', 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTAGATCTCGGTGGTCGCCGTATCATT',  # Read2的5'端适配器
#             '-o', fq1_val_path,  # 修剪后的Read1输出文件
#             '-p', fq2_val_path,  # 修剪后的Read2输出文件
#             '--pair-filter=any',  # 如果任一读段被修剪，则保留配对读段
#             '-e', '0.1',  # 允许的最大错误率
#             fq1_path,  # Read1的输入文件
#             fq2_path   # Read2的输入文件
#         ]

#         # 执行 cutadapt 命令
#         try:
#             process = subprocess.run(cutadapt_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#             print(process.stdout.decode())
#             print(process.stderr.decode())
#         except subprocess.CalledProcessError as e:
#             print(f"cutadapt command failed with error: {e}")
#         except Exception as e:
#             print(f"An unexpected error occurred: {e}")

# trim_galore
with open('file_all', 'r') as f:
    for line in f:
        fq1, fq2 = line.strip().split('\t')
        fq1_path = os.path.join(work_dir, fq1)
        fq2_path = os.path.join(work_dir, fq2)

        command = f'trim_galore -q 20 --phred33 --length 20 -e 0.1 --stringency 3 --paired --gzip -o {output_dir} {fq1_path} {fq2_path}'

        try:
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout, stderr = process.communicate()
            print(stdout.decode())
            print(stderr.decode())
        except subprocess.CalledProcessError as e:
            print(f"trim_galore command failed with error: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")


## 单端
# fq1_files = [file for file in os.listdir(work_dir) if 'EcST1.fq.gz' in file]
# # 生成文件列表
# file_all = '\n'.join(fq1 for fq1 in fq1_files)
# with open('file_all', 'w') as f:
#     f.write(file_all)

# # 处理每个 _R1.fq.gz 文件
# with open('file_all', 'r') as f:
#     for line in f:
#         fq1 = line.strip()
#         fq1_path = os.path.join(work_dir, fq1)

#         # 构建 trim_galore 命令
#         command = f'trim_galore -q 20 --phred33 --length 20 -e 0.1 --stringency 3 --gzip -o {output_dir} {fq1_path}'    ## --length 20

#         try:
#             # 执行命令
#             process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#             stdout, stderr = process.communicate()
#             print(stdout.decode())
#             print(stderr.decode())
#         except subprocess.CalledProcessError as e:
#             print(f"trim_galore command failed with error: {e}")
#         except Exception as e:
#             print(f"An unexpected error occurred: {e}")




igzip command line interface 2.31.0

Multicore support not enabled. Proceeding with single-core trimming.
Path to Cutadapt set as: 'cutadapt' (default)
Cutadapt seems to be working fine (tested command 'cutadapt --version')
Cutadapt version: 4.9
single-core operation.
igzip detected. Using igzip for decompressing

Output will be written into the directory: /home/john/ZouLab_Code/RNASeq/TrimGaloreData/HeSR_val/


AUTO-DETECTING ADAPTER TYPE
Attempting to auto-detect adapter type from the first 1 million sequences of the first file (>> /home/john/ZouLab_Code/RNASeq/Rawdata/HeSR/HeSRD3_R1.fq.gz <<)

Found perfect matches for the following adapter sequences:
Adapter type	Count	Sequence	Sequences analysed	Percentage
Illumina	121814	AGATCGGAAGAGC	1000000	12.18
smallRNA	0	TGGAATTCTCGG	1000000	0.00
Nextera	0	CTGTCTCTTATA	1000000	0.00
Using Illumina adapter for trimming (count: 121814). Second best hit was smallRNA (count: 0)

Writing report to '/home/john/ZouLab_Code/RNASeq/TrimGaloreData/HeSR

In [1]:
## 构建索引
import os
import subprocess

# hisat2
# geneRef_dir = '/home/john/RNA-seq/RNASeqDemoData/reference/GRCh38/'
# geneRef_name = 'GRCh38.genome'
# geneRef_fa = geneRef_dir + geneRef_name + '.fa'
# command = ['hisat2-build', '-p', '4', geneRef_fa, geneRef_name]

# try:
#     process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
#     print(process.stdout.decode())
# except subprocess.CalledProcessError as e:
#     print(f"bwa index command failed with error: {e}")


# ## bwa

# ref_genome_path = '/home/john/ZouLab_Code/RNASeq/reference/miRNA/processed_sequences.fa'
# output_dir = '/home/john/ZouLab_Code/RNASeq/reference/GCF5845_bacter_bwa_index'
# os.makedirs(output_dir, exist_ok=True)
# command = ['bwa', 'index', ref_genome_path]

# try:
#     process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
#     print(process.stdout.decode())
# except subprocess.CalledProcessError as e:
#     print(f"bwa index command failed with error: {e}")


## star
# thread_count = 5
# star_command = [
#     'STAR',
#     '--runMode', 'genomeGenerate',
#     '--runThreadN', str(thread_count),
#     '--genomeDir', '/home/john/RNA-seq/RNASeqDemoData/reference/star_index',
#     '--genomeFastaFiles', '/home/john/RNA-seq/RNASeqDemoData/reference/GRCh38/GRCh38.genome.fa',
#     '--sjdbGTFfile', '/home/john/RNA-seq/RNASeqDemoData/reference/GRCh38/gencode.v41.annotation.gtf',
#     '--sjdbOverhang', '149'
# ]

# process = subprocess.run(star_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
# print(process.stdout.decode())
# print(process.stderr.decode())


## hisat2
geneRef_dir = '/home/john/ZouLab_Code/RNASeq/reference/miRNA/'
geneRef_name = 'processed_sequences'
geneRef_fa = geneRef_dir + geneRef_name + '.fa'
command = ['hisat2-build', '-p', '4', geneRef_fa, geneRef_name]

try:
    process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
    print(process.stdout.decode())
except subprocess.CalledProcessError as e:
    print(f"bwa index command failed with error: {e}")

Building DifferenceCoverSample
  Building sPrime
  Building sPrimeOrder
  V-Sorting samples
  V-Sorting samples time: 00:00:00
  Allocating rank array
  Ranking v-sort output
  Ranking v-sort output time: 00:00:00
  Invoking Larsson-Sadakane on ranks
  Invoking Larsson-Sadakane on ranks time: 00:00:00
  Sanity-checking and returning
Building samples
Reserving space for 44 sample suffixes
Generating random suffixes
QSorting 44 sample offsets, eliminating duplicates
QSorting sample offsets, eliminating duplicates time: 00:00:00
Multikey QSorting 44 samples
  (Using difference cover)
  Multikey QSorting samples time: 00:00:00
Calculating bucket sizes
Splitting and merging
  Splitting and merging time: 00:00:00
Split 7, merged 20; iterating...
Splitting and merging
  Splitting and merging time: 00:00:00
Split 3, merged 5; iterating...
Splitting and merging
  Splitting and merging time: 00:00:00
Split 1, merged 1; iterating...
Splitting and merging
  Splitting and merging time: 00:00:00
Spl

# alignment (hisat2 / star / bwa)

In [13]:
import subprocess

cleaned_fq_dir = '/home/john/ZouLab_Code/RNASeq/TrimGaloreData/HPAdata/'
outputSam_dir = '/home/john/ZouLab_Code/RNASeq/samData/sam_HPA_GRCh38p14/'
outputLog_dir = '/home/john/ZouLab_Code/RNASeq/samData/sam_HPA_GRCh38p14/SamLog/'
# sampleAll = ['HeSRA1', 'HeSRA2', 'HeSRA3', 'HeSRD1', 'HeSRD2', 'HeSRD3', 'HeSRT1', 'HeSRT2', 'HeSRT3']
# 'H4RIP-2', 'H4RIP-3', 'H4RIP-5', 'H4RIP-6', 'H4RIP-7'
sampleAll = ['HPAA1', 'HPAA2', 'HPAA3', 'HPAD1', 'HPAD2', 'HPAD3', 'HPAT1', 'HPAT2', 'HPAT3']
## hisat2 双端
for sample in sampleAll:
    command = [
        'hisat2',
        '-p', '10',
        '-x', '/home/john/ZouLab_Code/RNASeq/reference/hisat2index/GRCh38.p14.genome', #'/home/john/RNA-seq/RNASeqDemoData/reference/ht_index/GRCh38.genome'
        '-1', cleaned_fq_dir + sample + '_R1_val_1.fq.gz',
        '-2', cleaned_fq_dir + sample + '_R1_val_1.fq.gz',
        '-S', outputSam_dir + sample + '.sam'
    ]

    output_log = outputLog_dir + sample + '.log'

    with open(output_log, 'w') as log:
        try:
            result = subprocess.run(command, stdout=subprocess.PIPE, stderr=log, check=True, text=True)
            print("Hisat2 success")
            print(result)
        except subprocess.CalledProcessError as e:
            print(f"Hisat2 failed with error: {e}")


# hisat2单端
# for sample in sampleAll:
#     command = [
#         'hisat2',
#         '-p', '5',  # 指定使用10个线程
#         '-x', '/home/john/ZouLab_Code/RNASeq/reference/GCF5845_bacter_index/GCF_000005845.2_ASM584v2_genomic', #'/home/john/RNA-seq/RNASeqDemoData/reference/ht_index/GRCh38.genome'
#         '-U', cleaned_fq_dir + sample + '_trimmed.fq.gz',
#         '-S', outputSam_dir + sample + '.sam'
#     ]

#     output_log = outputLog_dir + sample + '.log'

#     with open(output_log, 'w') as log:
#         try:
#             result = subprocess.run(command, stdout=subprocess.PIPE, stderr=log, check=True, text=True)
#             print("Hisat2 success")
#             print(result)
#         except subprocess.CalledProcessError as e:
#             print(f"Hisat2 failed with error: {e}")


## bwa

# cleaned_fq_dir = '/home/john/ZouLab_Code/RNASeq/TrimGaloreData/HESR_TX114_cleanedData_Bw/'
# outputSam_dir = '/home/john/ZouLab_Code/RNASeq/samData/sam_bwa_HESR_TX114_cleanedData_Bw_GCF/'
# outputLog_dir = '/home/john/ZouLab_Code/RNASeq/samData/sam_bwa_HESR_TX114_cleanedData_Bw_GCF/SamLog/'
# sampleAll = ['BwA3', 'BwD2', 'BwT1']
# for sample in sampleAll:
#     command = [
#         'bwa', 'mem',
#         '-t', '5',  ## 5线程
#         '-M', 
#         '/home/john/ZouLab_Code/RNASeq/reference/GCF5845_bacter_bwa_index/GCF_000005845.2_ASM584v2_genomic.fna',
#         cleaned_fq_dir + sample + '_R1_val_1.fq.gz',
#         cleaned_fq_dir + sample + '_R2_val_2.fq.gz',
#         ]
#     with open(outputSam_dir + sample + '.bwa.sam', 'w') as output_file:
#         try:
#             subprocess.run(command, stdout=output_file, stderr=subprocess.PIPE, check=True)
#             print("bwa success")
#         except subprocess.CalledProcessError as e:
#             print(f"bwa failed with error: {e}")


Hisat2 success
CompletedProcess(args=['hisat2', '-p', '10', '-x', '/home/john/ZouLab_Code/RNASeq/reference/hisat2index/GRCh38.p14.genome', '-1', '/home/john/ZouLab_Code/RNASeq/TrimGaloreData/HPAdata/HPAA1_R1_val_1.fq.gz', '-2', '/home/john/ZouLab_Code/RNASeq/TrimGaloreData/HPAdata/HPAA1_R1_val_1.fq.gz', '-S', '/home/john/ZouLab_Code/RNASeq/samData/sam_HPA_GRCh38p14/HPAA1.sam'], returncode=0, stdout='')
Hisat2 success
CompletedProcess(args=['hisat2', '-p', '10', '-x', '/home/john/ZouLab_Code/RNASeq/reference/hisat2index/GRCh38.p14.genome', '-1', '/home/john/ZouLab_Code/RNASeq/TrimGaloreData/HPAdata/HPAA2_R1_val_1.fq.gz', '-2', '/home/john/ZouLab_Code/RNASeq/TrimGaloreData/HPAdata/HPAA2_R1_val_1.fq.gz', '-S', '/home/john/ZouLab_Code/RNASeq/samData/sam_HPA_GRCh38p14/HPAA2.sam'], returncode=0, stdout='')
Hisat2 success
CompletedProcess(args=['hisat2', '-p', '10', '-x', '/home/john/ZouLab_Code/RNASeq/reference/hisat2index/GRCh38.p14.genome', '-1', '/home/john/ZouLab_Code/RNASeq/TrimGaloreDa

# transf sam to bam (samtools for bwa and hisat2)

In [14]:
## sam -> bam
import subprocess
import os

def run_command(command, stdout=subprocess.PIPE):
    try:
        result = subprocess.run(command, check=True, stdout=stdout, stderr=subprocess.PIPE)
        print(result)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Command '{' '.join(command)}' failed with error: {e.stderr.decode()}")
        return False

def sam_to_bam(input_sam, output_bam):
    command = [
        'samtools', 'sort',
        '-O', 'bam',
        '-@', '10',
        '-o', output_bam,
        input_sam
    ]
    return run_command(command)

def index_bam(bam_file):
    command = ['samtools', 'index', bam_file]
    return run_command(command)

def flagstat_bam(bam_file):
    command = ['samtools', 'flagstat', '-@', '10', bam_file]
    flagstat_file = bam_file.replace('.bam', '.flagstat')
    with open(flagstat_file, 'w') as output_file:
        return run_command(command, stdout=output_file)

input_dir = '/home/john/ZouLab_Code/RNASeq/samData/sam_HPA_GRCh38p14/'
output_dir = '/home/john/ZouLab_Code/RNASeq/bamData/bam_HPA_GRCh38p14/'
os.makedirs(output_dir, exist_ok=True)

for file_name in os.listdir(input_dir):
    if file_name.endswith('.sam'):
        input_sam = os.path.join(input_dir, file_name)
        output_bam = os.path.join(output_dir, file_name.replace('.sam', '.bam'))
        # sam_to_bam(input_sam, output_bam)

        if sam_to_bam(input_sam, output_bam):
            index_bam(output_bam)
            flagstat_bam(output_bam)

## 这一步结束后 需要到flagstat所在的路径下，通过  multiqc ./  进行质控可视化
## 可以通过 samtools view -h SRR1039508.bam | less 查看sort后的数据

CompletedProcess(args=['samtools', 'sort', '-O', 'bam', '-@', '10', '-o', '/home/john/ZouLab_Code/RNASeq/bamData/bam_HPA_GRCh38p14/HPAA2.bam', '/home/john/ZouLab_Code/RNASeq/samData/sam_HPA_GRCh38p14/HPAA2.sam'], returncode=0, stdout=b'', stderr=b'[bam_sort_core] merging from 40 files and 10 in-memory blocks...\n')
CompletedProcess(args=['samtools', 'index', '/home/john/ZouLab_Code/RNASeq/bamData/bam_HPA_GRCh38p14/HPAA2.bam'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['samtools', 'flagstat', '-@', '10', '/home/john/ZouLab_Code/RNASeq/bamData/bam_HPA_GRCh38p14/HPAA2.bam'], returncode=0, stderr=b'')
CompletedProcess(args=['samtools', 'sort', '-O', 'bam', '-@', '10', '-o', '/home/john/ZouLab_Code/RNASeq/bamData/bam_HPA_GRCh38p14/HPAT2.bam', '/home/john/ZouLab_Code/RNASeq/samData/sam_HPA_GRCh38p14/HPAT2.sam'], returncode=0, stdout=b'', stderr=b'[bam_sort_core] merging from 40 files and 10 in-memory blocks...\n')
CompletedProcess(args=['samtools', 'index', '/home/john/Zou

# 基因表达定量分析 (linux 生成表达矩阵)

In [ ]:
## 转录本组装预测
# stringtie -p 3 -e -G /home/cyh/Desktop/hugene_dir/GCF_000001405.40_GRCh38.p14_genomic.gff -o ly1.gtf -i /home/cyh/Desktop/his_result_sample1/sample1_sorted.bam

In [85]:
## 基因表达定量分析 (linux 生成表达矩阵) 
## 双端加 -p
# featureCounts -T 5 -p -t exon -g gene_id -a $"/home/john/RNA-seq/RNASeqDemoData/reference/GRCh38/gencode.v41.annotation.gtf" -o Countstable.txt *.bam 1>counts.id.log 2>&1 &
# GRCh38双端 : featureCounts -T 5 -p -t exon -g gene_id -a $"/home/john/ZouLab_Code/RNASeq/reference/GRCh38.p14/gencode.v46.annotation.gtf" -o Countstable.txt *.bam 1>counts.id.log 2>&1 &
# GRCh38单端 : featureCounts -T 5 -t exon -g gene_id -a $"/home/john/ZouLab_Code/RNASeq/reference/GRCh38.p14/gencode.v46.annotation.gtf" -o Countstable.txt *.bam 1>counts.id.log 2>&1 &
# tRNA : featureCounts -T 5 -p -t exon -g gene_id -a $"/home/john/ZouLab_Code/RNASeq/reference/tRNA/tRNA_annotation.gtf" -o Countstable.txt *.bam 1>counts.id.log 2>&1 &
# miRNA : featureCounts -T 5 -t miRNA -g ID -F GFF3 -a $"/home/john/ZouLab_Code/RNASeq/reference/miRNA/miRNA_hsa.gff3" -o Countstable.txt *.bam 1>counts.id.log 2>&1 &
#featureCounts -T 5 -p -t exon -g gene_id -a $"/home/john/ZouLab_Code/RNASeq/reference/GCF5845_bacter/genomic.gtf" -o Countstable.txt *.bam 1>counts.id.log 2>&1 &
# cd /home/john/RNA-seq/RNASeqDemoData/featureCount
# multiqc all.id.txt.summary
# multiqc Countstable.txt.summary

import pandas as pd

prefix_dir = "/home/john/ZouLab_Code/RNASeq/featureCountsData/FC_4HNE-RIPseq_cleanedData/"
countstable = "/home/john/ZouLab_Code/RNASeq/featureCountsData/FC_4HNE-RIPseq_cleanedData/Countstable.txt"

df = pd.read_csv(countstable, sep="\t", header=0)
df.set_index(df.columns[0], inplace=True)
df = df.iloc[:, 0:]
df.to_csv(prefix_dir + "Countstable.csv")

## 只取featureCounts
prefix_name = "QuantAnalGeneExpression"
prefix = prefix_dir + prefix_name

featureCounts_meta = df.iloc[:, 5:]
featureCounts_df = featureCounts_meta[featureCounts_meta.sum(axis=1) > 0]
featureCounts_df.to_csv(f"{prefix_dir}FC_filted.csv")


# TPM计算
print(featureCounts_meta.shape[0])
kb = featureCounts_meta.shape[0] / 1000
rpk = featureCounts_df / kb
tpm = (rpk / rpk.sum()) * 1e6
tpm.to_csv(f"{prefix}_tpm.csv", index_label="Geneid")

# # FPKM计算
fpkm = (rpk / featureCounts_df.sum()) * 1e6
fpkm.to_csv(f"{prefix}_fpkm.csv", index_label="Geneid")


63086


In [ ]:
## 基因表达定量分析 (linux 生成表达矩阵)
# featureCounts -T 5 -p -t exon -g gene_id -a $"/home/john/RNA-seq/RNASeqDemoData/reference/GRCh38/gencode.v41.annotation.gtf" -o Countstable.txt *.bam 1>counts.id.log 2>&1 &
# cd /home/john/RNA-seq/RNASeqDemoData/featureCount
# multiqc all.id.txt.summary

import pandas as pd
countstable = "/home/john/ZouLab_Code/RNASeq/featureCountsData/FC_4HNE-RIPseq_cleanedData/Countstable.txt"
csv_output = "/home/john/ZouLab_Code/RNASeq/featureCountsData/FC_4HNE-RIPseq_cleanedDataCountstable.csv"
df = pd.read_csv(countstable, sep="\t", header=0)
df.to_csv(csv_output, index=False)

## 只取featureCounts
prefix_dir = "/home/john/ZouLab_Code/RNASeq/featureCountsData/FC_4HNE-RIPseq_cleanedData/"
prefix_name = "QuantAnalGeneExpression"
prefix = prefix_dir + prefix_name

Geneid = df.iloc[:, 0:1]
DEGNum = df.iloc[:, 6:]
featureCounts_meta = pd.concat([Geneid, DEGNum], axis=1)
featureCounts_df = featureCounts_meta[featureCounts_meta.iloc[:, 1:].sum(axis=1) > 0]
featureCounts_df.to_csv(f"{prefix_dir}FC_filted.csv")

## TPM计算
kb = featureCounts_meta.shape[0] / 1000
rpk = featureCounts_df / kb
tpm = (rpk / rpk.sum()) * 1e6
tpm.to_csv(f"{prefix}_tpm.csv", index_label="Geneid")

## FPKM计算
fpkm = (rpk / featureCounts_df.sum()) * 1e6
fpkm.to_csv(f"{prefix}_fpkm.csv", index_label="Geneid")
